## exe-1) 유방암 양성/음성 여부 판단을 위한 모델 선정

## 데이터 불러오고 독립변수, 종속변수 생성 및 정규화까지

## 5시 40분까지 제출 : 파일명 끝에 본인 이름
##### - 구글드라이브 > 산출물 > 머신러닝 > 03_유방암_판별모델생성
##### (모든 모델 확인 후 합당한 모델 선정한 [해석] 포함)

In [1]:
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer

breast_cancer_data = load_breast_cancer()
#breast_cancer_data

In [3]:
import pandas as pd
df = pd.DataFrame(data = breast_cancer_data.data, columns = breast_cancer_data.feature_names)
df['target'] = pd.Series(breast_cancer_data.target)

In [4]:
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [5]:
df_x = df.iloc[:,:-1]
df_y = df.loc[:,'target']

In [6]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = \
      train_test_split(df_x, df_y, test_size= 0.25 , random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input,train_target)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

## 앙상블 - 교차검증 모델 - 랜덤포레스트

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
# 랜덤포레스트 객체생성 : 코어 모두 사용
rf = RandomForestClassifier(n_jobs = -1 , random_state = 42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

# 교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(rf, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv=splitter)

# 최종 훈련평가 결과 및 검증결과
scores
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

훈련 score =  1.0 테스트 score=  0.9694352159468439


In [21]:
rf.fit(train_scaled, train_target)
# 특성 중요도 조회하기
rf.feature_importances_
rf.score(test_scaled, test_target)

0.965034965034965

In [10]:
### oob : 기능 사용
# - 훈련에 참여하지 못한 잔여샘플 사용하는 기능
# 기본은 사용안함,
# 기본은 사용안함
# 훈련한번더하고 검증까지
rf = RandomForestClassifier(oob_score = True, n_jobs = -1 , random_state=42)
rf.fit(train_scaled, train_target)
print(rf.oob_score_)

0.960093896713615


### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## 엑스트라 트리 - 교차검증

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

# 교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(et, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv=splitter)

## 최종 및 훈련평가 결과 및 검증결과
scores
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

훈련 score =  1.0 테스트 score=  0.9741971207087486


In [12]:
et.fit(train_scaled,train_target)
et.feature_importances_
et.score(test_scaled, test_target)

0.9790209790209791

### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## 그래디언트 부스트 - 교차검증

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

scores = cross_validate(gb, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv = splitter)

print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))


훈련 score =  1.0 테스트 score=  0.9647840531561462


In [14]:
gb.fit(train_scaled, train_target)
print(et.feature_importances_)
gb.score(test_scaled, test_target)

[0.06526478 0.02263436 0.04037335 0.04137443 0.01127867 0.03141925
 0.06792653 0.07436193 0.00680299 0.00864427 0.01592531 0.00632018
 0.01699375 0.0355914  0.00717712 0.0077469  0.00781951 0.01101074
 0.00687928 0.00718805 0.10172404 0.02771603 0.10386939 0.06440105
 0.01958991 0.02389705 0.05887149 0.08507095 0.01407766 0.00804961]


0.958041958041958

### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## 히스토그램기반 그래디언트부스팅 분류 모델

In [15]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state = 42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)


scores = cross_validate(hgb, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv = splitter)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

훈련 score =  1.0 테스트 score=  0.974141749723145


In [16]:
hgb.fit(train_scaled, train_target)
hgb.score(test_scaled, test_target)

0.951048951048951

### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## XGBoost - 히스토그램기반

In [17]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method="hist", random_state=42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

scores = cross_validate(xgb, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv = splitter)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

C:\Users\admin\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


훈련 score =  1.0 테스트 score=  0.9717607973421927


In [18]:
xgb.fit(train_scaled,train_target)
xgb.score(test_scaled, test_target)

[16:26:40] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\admin\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.958041958041958

### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## LightGBM

In [19]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=42)
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

scores = cross_validate(lgbm, train_scaled, train_target, return_train_score = True, n_jobs = -1, cv = splitter)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

훈련 score =  1.0 테스트 score=  0.9694905869324474


In [20]:
lgbm.fit(train_scaled, train_target)
lgbm.score(test_scaled, test_target)

0.958041958041958

### 해석
###### 훈련이 1.0이 나오므로 수치는 높으나 연구에서 사용불가한 훈련성능을 나타내었다
###### 테스트데이터와의 격차가 0.1 미만이고 훈련성능이 더 좋으므로 과적합은 아니다

## 의사결정나무 - GridSerchCV

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'max_depth' : range(5, 20, 1)}
splitter = StratifiedKFold(n_splits = 7, shuffle = True, random_state = 42)

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params,  cv = splitter, n_jobs = -1)

gs.fit(train_scaled, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [29]:
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=6, random_state=42)


In [32]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt= gs.best_estimator_
print(dt.score(train_scaled, train_target))

# 이렇게하는거 아니었어?
print(gs.score(test_scaled,test_target))

0.9976525821596244
0.951048951048951


In [34]:
# gird serch가 찾은 모든 값
# 딕셔너리의 key값만 조회하기
# print(gs.cv_results_.keys())
### params 값 확인
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


## 이미지분류 이야기한다고 여기까지밖에 못했습니다 ㅠㅠ